In [5]:
import open3d as o3d

In [6]:
pcd = o3d.io.read_point_cloud("viewPoints.pcd")

In [7]:
# Voxel downsampling
voxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.1)

    # Statistical outlier removal
clean_pcd, ind = voxel_down_pcd.remove_statistical_outlier(nb_neighbors=50, std_ratio=1.0)

    # Pass-through filter (assuming filtering on z-axis)
pass_through_filter = clean_pcd.crop(
        o3d.geometry.AxisAlignedBoundingBox(min_bound=(-float('inf'), -float('inf'), 0.0),
                                            max_bound=(float('inf'), float('inf'), 1.5)))

    # Save the final point cloud
o3d.io.write_point_cloud("filtered_point_cloud.pcd", pass_through_filter)

print("Filtering complete. The filtered point cloud is saved as 'filtered_point_cloud.pcd'.")

Filtering complete. The filtered point cloud is saved as 'filtered_point_cloud.pcd'.


In [8]:
o3d.visualization.draw_geometries([pass_through_filter])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [19]:
import open3d as o3d
import numpy as np

def apply_frustum_culling(point_cloud, camera_pose, fov, near_plane, far_plane):
    print("\n\nApplying Filter")

    # Convert FOV from degrees to radians and calculate half-angle
    fov_rad = np.deg2rad(fov)
    half_angle = fov_rad / 2

    # Calculate frustum bounds
    points = np.asarray(point_cloud.points)
    directions = points - camera_pose[:3, 3]  # vector from camera to point
    distances = np.linalg.norm(directions, axis=1)
    directions_normalized = directions / distances[:, np.newaxis]

    # Calculate angle with the camera forward vector (z-axis)
    cos_angle = np.dot(directions_normalized, camera_pose[:3, 2])
    angles = np.arccos(np.clip(cos_angle, -1.0, 1.0))  # Clipping values to valid range for acos

    # Filter points within frustum
    mask = (angles < half_angle) & (distances > near_plane) & (distances < far_plane)
    filtered_points = points[mask]
    print(f"Number of points before filtering: {len(points)}")
    print(f"Number of points after filtering: {len(filtered_points)}")

    filtered_cloud = o3d.geometry.PointCloud()
    filtered_cloud.points = o3d.utility.Vector3dVector(filtered_points)

    return filtered_cloud

def main():
    # Load a point cloud
    cloud = o3d.io.read_point_cloud("viewPoints.pcd")

    # Compute centroid and set the camera to look at the centroid
    centroid = cloud.get_center()
    camera_pose = np.eye(4)
    camera_pose[:3, 3] = centroid + np.array([0, 0, -10])  # Position camera 10 units along the z-axis from the centroid

    # Frustum parameters
    fov = 100  # Wide field of view
    near_plane = 1.0  # Near clipping plane
    far_plane = 1000  # Far clipping plane

    # Apply frustum culling
    filtered_cloud = apply_frustum_culling(cloud, camera_pose, fov, near_plane, far_plane)

    # Save the filtered point cloud
    if not filtered_cloud.is_empty():
        o3d.io.write_point_cloud("filtered_viewPoints.pcd", filtered_cloud)
        print("Filtering complete. The filtered point cloud is saved as 'filtered_viewPoints.pcd'.")
    else:
        print("No points left after filtering; check the camera pose and frustum parameters.")
    
    return filtered_cloud

if __name__ == "__main__":
    x=main()




Applying Filter
Number of points before filtering: 112766
Number of points after filtering: 62199
Filtering complete. The filtered point cloud is saved as 'filtered_viewPoints.pcd'.


In [ ]:
o3d.visualization.draw_geometries([x])